<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [11]:
!pip install pycuda


     |████████████████████████████████| 1.6MB 9.0MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=d87c00e15ccf78d878c6d02fdd84901a076392de44bed48204c41232eefaf0da
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=11d85f5281b8309be4b3546409fabf70305ee4b5f14bfa076da35ded33f5af02
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

##3.1 Desarrollo en CPU

In [5]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  5#@param {type: "number"}
orden_taylor = 10#@param {type: "number"}
x0 =  3#@param {type: "number"}
x = 4#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import math

def factorial(x):
  if x == 0 or x == 1:
    return 1
  factorial = 1
  for i in range (1, x+1):
    factorial = factorial * i
  return factorial

tiempoTotal = time.time() * 1000

#coeficientes = numpy.random.randint(10, size = orden_polinomio +1)
coeficientes = numpy.array([3, 2, 3, 4])

longitud_poli = len(coeficientes)

derivadas = numpy.arange(longitud_poli)

print("Coeficientes:\n", coeficientes)

polin_taylor = numpy.zeros(orden_taylor + 1 , dtype=float)
longitud_taylor = len(polin_taylor)
print("Orden polinomio", longitud_poli - 1 )
print("Longitud_Taylor:", longitud_taylor)
print("*****************************************************************")

tiempoBucle = time.time() *1000
#----------------------ALGORITMO----------------------------
for idx_t in range(0, longitud_taylor):
  valor = 0
  aux = 0
  
  if idx_t == 0:
    polin_taylor[idx_t] += coeficientes[idx_t]
    idx_p = idx_t + 1
    for idx in range(idx_p, longitud_poli):
      polin_taylor[idx_t] += coeficientes[idx] * math.pow(x0, derivadas[idx])
      print("idt:", idx_t, ":idx:", idx, "|", "aux:", polin_taylor[idx_t])
  else:
    if idx_t < longitud_poli - 1:
      aux += coeficientes[idx_t]
      idx_p = idx_t + 1
      resta = math.pow(x - x0, idx_t)
      f = factorial(idx_t)
      for idx in range(idx_p, longitud_poli):
        termino = coeficientes[idx] * derivadas[idx]
        coeficientes[idx] = termino
        derivadas[idx] = derivadas[idx] - 1
        factor = math.pow(x0, derivadas[idx])
        aux += termino * factor        
        print("idt:", idx_t, ":idx:", idx,"|",
              "aux:", aux , "termino:", termino)
      polin_taylor[idx_t] = (aux * resta) / f        
    else:
      if idx_t >= longitud_poli: 
        print("idx_t:", idx_t)
        valor =  0
        polin_taylor[idx_t] = valor
        print("Valor:", valor)
      else:
        print("idx_t:", idx_t)
        valor =  coeficientes[idx_t]
        resta = math.pow(x - x0, idx_t)
        f = factorial(idx_t)
        polin_taylor[idx_t] = (valor * resta)/ f
        print("Valor:", valor)
  print("-----------------------------------------------------------------")
#----------------------FINALIZACION----------------------------

tiempoBucle = (time.time() *1000) - tiempoBucle
print("Termino el algoritmo")
print(polin_taylor)

tiempoTotal = (time.time() *1000) - tiempoTotal
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Coeficientes:
 [3 2 3 4]
Orden polinomio 3
Longitud_Taylor: 11
*****************************************************************
idt: 0 :idx: 1 | aux: 9.0
idt: 0 :idx: 2 | aux: 36.0
idt: 0 :idx: 3 | aux: 144.0
-----------------------------------------------------------------
idt: 1 :idx: 2 | aux: 20.0 termino: 6
idt: 1 :idx: 3 | aux: 128.0 termino: 12
-----------------------------------------------------------------
idt: 2 :idx: 3 | aux: 78.0 termino: 24
-----------------------------------------------------------------
idx_t: 3
Valor: 24
-----------------------------------------------------------------
idx_t: 4
Valor: 0
-----------------------------------------------------------------
idx_t: 5
Valor: 0
-----------------------------------------------------------------
idx_t: 6
Valor: 0
-----------------------------------------------------------------
idx_t: 7
Valor: 0
-----------------------------------------------------------------
idx_t: 8
Valor: 0
------------------------------------

##3.2 Desarrollo en GPU

In [4]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio = 50000#@param {type: "number"}
orden_taylor = 4#@param {type: "number"}
x0 = 3#@param {type: "number"}
x = 1#@param {type: "number"}
# --------------------------------------------

import numpy
import time

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time() * 1000)

# --------------------------------------------
# CPU - Defino la memoria de los vectores en cpu.
#coeficientes_cpu = numpy.random.randn(orden_polinomio + 1)
coeficientes_cpu = numpy.array([3, 2, 3, 4])
coeficientes_cpu = coeficientes_cpu.astype(numpy.float32())
long_pol = len(coeficientes_cpu)

taylor_cpu = numpy.zeros(orden_taylor + 1, dtype=float)
taylor_cpu = taylor_cpu.astype(numpy.float32())
long_tay = len(coeficientes_cpu)

derivadas_cpu = numpy.arange(long_pol)
derivadas_cpu = derivadas_cpu.astype(numpy.int32)

resultado_cpu = numpy.empty_like(taylor_cpu)
print("C:", coeficientes_cpu)
print("T:", taylor_cpu)
print("D:", derivadas_cpu)
print("R:", resultado_cpu)

# CPU - reservo la memoria GPU.
coeficientes_gpu = cuda.mem_alloc( coeficientes_cpu.nbytes )
taylor_gpu = cuda.mem_alloc( taylor_cpu.nbytes )
derivadas_gpu = cuda.mem_alloc( derivadas_cpu.nbytes )
# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( coeficientes_gpu, coeficientes_cpu )
cuda.memcpy_htod( taylor_gpu, taylor_cpu )
cuda.memcpy_htod( derivadas_gpu, derivadas_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor(  int long_p, int long_t,
                                float x0, float x,
                                float* C, float* T, float* D){
  
  int idx_t = threadIdx.x + blockIdx.x*blockDim.x;

  int f = 1;
  int i;
  int idx;
  for(i = 1; i <= idx_t; i++){
    f*= i;
  }
  
  float aux = 0;
  float resta = 0;
  if( idx_t == 0){
    T[idx_t] = 0;
    T[idx_t] += C[idx_t];
    int idx_p = idx_t + 1;
    for(idx = idx_p; idx < long_p; idx++){
      T[idx_t] += C[idx] * pow((double) x0, (double)idx);
    } else {
      if(idx_t < long_p -1){
        aux += C[idx_t]
        idx_p = idx_t + 1
        resta = pow((double)x-x0, (double)idx_t);
        for(idx = idx_p, idx < long_p; idx++){
          termino = C[idx] * D[idx];
          
          aux += termino * factor;
        }
        T[idx_t] = (aux * resta) / f
      } else {
        if(idx_t >= long_p){
          T[idx_t] = 0;
        } else{
          valor = C[idx_t];
          resta = pow((double)x-x0, idx_t);
          T[idx_t] = (valor * resta) / f;
        }
      }
    }
  }
}
""")

# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = time.time()*1000

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (long_tay+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(long_pol), numpy.int32(long_tay),
       numpy.float32(x0), numpy.float32(x),      
       coeficientes_gpu, taylor_gpu, derivadas_gpu,
       block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = time.time()*1000 - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( resultado_cpu, taylor_gpu)
print("Resultado: ", resultado_cpu)

tiempoTotal = time.time() *1000 - tiempoTotal
print("-----------------------------------------------")
print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempoTotal, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

C: [3. 2. 3. 4.]
T: [0. 0. 0. 0. 0.]
D: [0 1 2 3]
R: [12.  0.  0.  0.  0.]
Thread x:  256 , Bloque x: 1
Resultado:  [144.   0.   0.   0.   0.]
-----------------------------------------------
Thread x:  256 , Bloque x: 1
Tiempo CPU:  357.4736328125 [ms]
Tiempo GPU:  0.517578125 [ms]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(15): warning: variable "aux" was declared but never referenced

kernel.cu(16): warning: variable "resta" was declared but never referenced




#4. Tabla de pasos

#5. Conclusiones